In [1]:
import optuna
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Import and rearrange data

In [2]:
# Load data
bkgd = pd.read_hdf("data/preprocessed/bkgd.h5", key="bkgd")
vlq = pd.read_hdf("data/preprocessed/vlq.h5", key="vlq")
X_train = pd.concat([bkgd, vlq])
del bkgd, vlq

In [3]:
# Train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train.drop(["Label"], axis=1), X_train["Label"], 
                                                    test_size=1/3, random_state=56)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=60)

In [4]:
# Save train data
X_train.to_hdf("data/classifier/train.h5", key="X")
y_train.to_hdf("data/classifier/train.h5", key="y")

# Save validation data
X_val.to_hdf("data/classifier/validation.h5", key="X")
y_val.to_hdf("data/classifier/validation.h5", key="y")

# Save test data
X_test.to_hdf("data/classifier/test.h5", key="X")
y_test.to_hdf("data/classifier/test.h5", key="y")

In [5]:
# Get data samples
train_samples, val_samples, test_samples = X_train["Sample"], X_val["Sample"], X_test["Sample"]

# Get data weights
train_weights, val_weights, test_weights = X_train["gen_weights"], X_val["gen_weights"], X_test["gen_weights"]

# Remove sample and weight columns
X_train.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_val.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_test.drop(["Sample", "gen_weights"], axis=1, inplace=True)

# Calculate class weights
class_weights = {
    0: 1,
    1: len(y_train[y_train==0]) / len(y_train[y_train==1])
}

# Create model

In [6]:
def get_model(hidden_layers=[100, 100, 100], dropout=0.1, batch_norm=True, optimizer="Nadam"):
    """
    This function creates a keras model, given the desired hidden_layers, dropout rate
    and optimizer of choice
    
    hidden_layers -> [int]: size of each desired hidden layer
    dropout -> float: desired dropout rate
    optimizer -> string: optimizer you choose to utilize
    
    returns a keras model
    """
    
    # Generate model structure
    inputs = keras.Input(shape=(69,))
    bn = keras.layers.BatchNormalization()(inputs)
    drop = bn
    for i in range(len(hidden_layers)-1):
        fc = keras.layers.Dense(hidden_layers[i], activation='relu')(drop)
        if batch_norm:
            bn = keras.layers.BatchNormalization()(fc)
        else:
            bn = fc
        drop = keras.layers.Dropout(dropout)(bn, training=True)
    fc = keras.layers.Dense(hidden_layers[-1], activation='relu')(drop)
    outputs = keras.layers.Dense(1, activation='sigmoid')(fc)
    
    # Instanciate and compile model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss="binary_crossentropy",
                  metrics=["accuracy", keras.metrics.AUC()])
    model.summary()
    
    return model

# Optuna Bayesian Hyperparameter Search Function

In [7]:
def objective(trial):
    """
    Optuna objective function for bayesian inference hyperparameter search
    
    trial -> optuna trial object
    
    return -> float: validation accuracy of the best model with early stopping and model checkpoint
    """
    
    # Defining parameters
    num_layers = trial.suggest_int("num_hidden_layers", 1, 5)
    hidden_layers = []
    for i in range(num_layers):
        num_features = trial.suggest_int(f"num_features_layer_{i}", 20, 150)
        hidden_layers.append(num_features)
    dropout = trial.suggest_discrete_uniform("dropout", 0.05, 0.5, 0.01)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024])
    batch_norm = trial.suggest_categorical("batch_norm", [True, False])
    optimizer = "Adam"
    
    # Create model
    model = get_model(hidden_layers, dropout, batch_norm, optimizer)
    name = f"trial_{trial.number}"
    
    # Callbacks
    TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)
    ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, mode="min")
    MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")
    LR = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, mode="min", 
                                       min_lr=1e-6)
    
    # Train
    model.fit(X_train.values, y_train.values, batch_size=batch_size, epochs=500, callbacks=[TB, ES, MC, LR],
              validation_data=(X_val.values, y_val.values, val_weights.values), shuffle=True,
              sample_weight=train_weights.values, class_weight=class_weights, verbose=2)
    
    # Get accuracy
    y_preds = model.predict(X_val.values)
    val_accuracy = accuracy_score(y_preds.round(), y_val.values)
    
    return val_accuracy

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

0
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 69)                276       
_________________________________________________________________
dense (Dense)                (None, 76)                5320      
_________________________________________________________________
dropout (Dropout)            (None, 76)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 51)                3927      
_________________________________________________________________
dropout_1 (Dropout)          (None, 51)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 51)             

[I 2020-08-11 15:31:28,376] Trial 0 finished with value: 0.8468313687964856 and parameters: {'num_hidden_layers': 3, 'num_features_layer_0': 76, 'num_features_layer_1': 51, 'num_features_layer_2': 51, 'dropout': 0.15000000000000002, 'batch_size': 1024, 'batch_norm': False}. Best is trial 0 with value: 0.8468313687964856.


1
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 69)                276       
_________________________________________________________________
dense_4 (Dense)              (None, 77)                5390      
_________________________________________________________________
batch_normalization_2 (Batch (None, 77)                308       
_________________________________________________________________
dropout_2 (Dropout)          (None, 77)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 70)                5460      
_________________________________________________________________
batch_normalization_3 (Batch (None, 70)             

[I 2020-08-11 15:33:51,959] Trial 1 finished with value: 0.8209553013498035 and parameters: {'num_hidden_layers': 5, 'num_features_layer_0': 77, 'num_features_layer_1': 70, 'num_features_layer_2': 59, 'num_features_layer_3': 114, 'num_features_layer_4': 133, 'dropout': 0.37, 'batch_size': 512, 'batch_norm': True}. Best is trial 0 with value: 0.8468313687964856.


2
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 69)                276       
_________________________________________________________________
dense_10 (Dense)             (None, 68)                4760      
_________________________________________________________________
dropout_6 (Dropout)          (None, 68)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 49)                3381      
_________________________________________________________________
dropout_7 (Dropout)          (None, 49)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 67)             

...

[I 2020-08-11 16:00:51,384] Trial 17 finished with value: 0.9027916721035768 and parameters: {'num_hidden_layers': 4, 'num_features_layer_0': 142, 'num_features_layer_1': 99, 'num_features_layer_2': 130, 'num_features_layer_3': 96, 'dropout': 0.12000000000000001, 'batch_size': 256, 'batch_norm': True}. Best is trial 14 with value: 0.914027952967103.


18
Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
batch_normalization_35 (Batc (None, 69)                276       
_________________________________________________________________
dense_71 (Dense)             (None, 114)               7980      
_________________________________________________________________
batch_normalization_36 (Batc (None, 114)               456       
_________________________________________________________________
dropout_35 (Dropout)         (None, 114)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 134)               15410     
_________________________________________________________________
batch_normalization_37 (Batc (None, 134)          

[I 2020-08-11 16:03:00,737] Trial 18 finished with value: 0.9164383164426659 and parameters: {'num_hidden_layers': 3, 'num_features_layer_0': 114, 'num_features_layer_1': 134, 'num_features_layer_2': 28, 'dropout': 0.05, 'batch_size': 256, 'batch_norm': True}. Best is trial 18 with value: 0.9164383164426659.


19
Model: "functional_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
batch_normalization_38 (Batc (None, 69)                276       
_________________________________________________________________
dense_75 (Dense)             (None, 51)                3570      
_________________________________________________________________
dropout_37 (Dropout)         (None, 51)                0         
_________________________________________________________________
dense_76 (Dense)             (None, 127)               6604      
_________________________________________________________________
dropout_38 (Dropout)         (None, 127)               0         
_________________________________________________________________
dense_77 (Dense)             (None, 33)           

[I 2020-08-11 16:04:58,069] Trial 19 finished with value: 0.7402498078958433 and parameters: {'num_hidden_layers': 4, 'num_features_layer_0': 51, 'num_features_layer_1': 127, 'num_features_layer_2': 33, 'num_features_layer_3': 150, 'dropout': 0.48, 'batch_size': 256, 'batch_norm': False}. Best is trial 18 with value: 0.9164383164426659.


In [21]:
def process_study_df(study_df):
    """
    This function processes the study dataframe so that it is more concise and easier to read
    
    study_df -> pandas dataframe: optuna default study dataframe
    
    return -> pandas dataframe: processes study dataframe
    """
    
    study_df.rename(columns={"number": "trial", "value": "val_acc"}, inplace=True)
    study_df.drop(["datetime_start", "datetime_complete"], axis=1, inplace=True)
    param_name_dict = {}
    for column in study_df:
        if "params" in column:
            param_name_dict[column] = column.replace("params_", "")
    study_df.rename(columns=param_name_dict, inplace=True)
    
    return study_df

In [26]:
study_df = process_study_df(study.trials_dataframe())
study_df.to_hdf("optuna_studies/study_1.h5", key="study")
study_df

,trial,val_acc,duration,batch_norm,batch_size,dropout,num_features_layer_0,num_features_layer_1,num_features_layer_2,num_features_layer_3,num_features_layer_4,num_hidden_layers,state
0,0,0.846831,0 days 00:01:13.198452,False,1024,0.15,76,51.0,51.0,NaN,NaN,3,COMPLETE
1,1,0.820955,0 days 00:02:23.581527,True,512,0.37,77,70.0,59.0,114.0,133.0,5,COMPLETE
2,2,0.783027,0 days 00:01:28.820673,False,1024,0.23,68,49.0,67.0,56.0,NaN,4,COMPLETE
3,3,0.868666,0 days 00:00:54.362678,True,512,0.41,59,NaN,NaN,NaN,NaN,1,COMPLETE
4,4,0.885796,0 days 00:01:08.137980,False,512,0.10,92,130.0,NaN,NaN,NaN,2,COMPLETE
5,5,0.840191,0 days 00:01:00.819084,False,1024,0.40,125,149.0,NaN,NaN,NaN,2,COMPLETE
6,6,0.807001,0 days 00:03:27.648310,True,1024,0.50,94,123.0,106.0,41.0,NaN,4,COMPLETE
7,7,0.740250,0 days 00:02:00.323031,False,1024,0.34,128,23.0,65.0,39.0,104.0,5,COMPLETE
8,8,0.874708,0 days 00:00:56.825841,True,512,0.19,129,NaN,NaN,NaN,NaN,1,COMPLETE
9,9,0.875556,0 days 00:01:10.258657,True,512,0.18,99,37.0,NaN,NaN,NaN,2,COMPLETE
